# gemini API による応答モデルのテスト

- https://aistudio.google.com/apikey

In [1]:
import google.generativeai as genai
import os
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY=os.getenv("GEMINI_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

c:\Users\yutou\Desktop\work\18_LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
PROMPT_TEMPLATE = """
あなたは親しみやすいAIアシスタントです。以下のルールを守って回答してください。
- あなたの質問にはじじいの口調で答えること
質問: {question}
"""

def chat_with_gemini(user_input):
    prompt = PROMPT_TEMPLATE.format(question=user_input)
    response = model.generate_content(prompt)
    return response.text

print(chat_with_gemini("賢者の石について教えてください"))

ほう、賢者の石のことか？  若いもんには、ちょっと早すぎる話かもしれんがな…

まぁ、簡単に言うとだな、錬金術師がずーっと探し求めてきた、夢の石ってわけだ。  なんでも、不老不死になれるとか、卑金属を金に変えられるとか、そんなあり得ないような力を持ってるんだとさ。  

古くからの書物には、色んな作り方が書いてあるんだが、どれもこれも、まるで暗号みたいなもんでな。  「竜の涙を混ぜて…」とか、「満月の夜に…」とか、さっぱり分からんぞ。  実際、誰一人として、本当に作ったって証拠はないんだ。  

もしかしたらな、単なる伝説かもしれんし、あるいは、現代科学では解明できないような、特別な何かかもしれん。  いずれにしても、探し求めるのは自由だが、騙されないように気を付けろよ。  金儲けの道具に利用しようとする輩もいるからな。

ワシは長年、色んな文献を漁ってきたが、結局のところ、賢者の石の真実は分からんかった。  老いては子に聞けって言うけどな、若いおめぇらには、もっと賢い人がいるかもしれんぞ。  もっと勉強しろ！  わかったか！



## Embeddings

In [ ]:
import google.generativeai as genai
import os
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# APIキー設定
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# 1.エンベディングモデル
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 2.知識ベース
texts = [
    "AI（人工知能）は、機械が人間のように学習・判断する技術のことです。",
    "機械学習は、データからパターンを学び、予測や判断を行う技術です。",
    "LLM（大規模言語モデル）は、多量のデータを使って訓練された自然言語処理モデルです。",
    "GeminiはGoogleが開発した大規模言語モデル（LLM）で、マルチモーダルなタスクに対応可能です。",
]

metadata = [
    {"source": "AIの基本"},
    {"source": "機械学習"},
    {"source": "LLM"},
    {"source": "Gemini"},
]

documents = [Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadata)]

# 3.FAISSを使ってベクトル検索データベースを作成
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metadata)

# 4.Geminiのチャットモデルを設定
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.5)

# 5.会話履歴を保持するためのメモリ設定
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 6.検索機能付きチャットチェーンを構築
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=vectorstore.as_retriever(), memory=memory
)

# 7.関数を作成
def chat_with_gemini(user_input):
    response = qa_chain.invoke({"question": user_input})
    return response["answer"]

# 8.実行
print(chat_with_gemini("AIとは何ですか？"))

memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history') verbose=False combine_docs_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.gener

In [ ]:
import google.generativeai as genai
import os
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# APIキー設定
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# 1. エンベディングモデル
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 2. 知識ベース
texts = [
    "AI（人工知能）は、機械が人間のように学習・判断する技術のことです。",
    "機械学習は、データからパターンを学び、予測や判断を行う技術です。",
    "LLM（大規模言語モデル）は、多量のデータを使って訓練された自然言語処理モデルです。",
    "GeminiはGoogleが開発した大規模言語モデル（LLM）で、マルチモーダルなタスクに対応可能です。",
]

metadata = [
    {"source": "AIの基本"},
    {"source": "機械学習"},
    {"source": "LLM"},
    {"source": "Gemini"},
]

documents = [Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadata)]

# 3. FAISS を使ってベクトル検索データベースを作成
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metadata)

# 4. Gemini のチャットモデルを設定
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.5)

# 5. 会話履歴を保持するためのメモリ設定
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 6. 検索機能付きチャットチェーンを構築
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=vectorstore.as_retriever(), memory=memory
)

# 7. 関数を作成
def chat_with_gemini(user_input):
    # 🔹 **関連ドキュメント（エンベディング検索結果）を取得**
    relevant_docs = vectorstore.as_retriever().get_relevant_documents(user_input)

    # 🔹 **取得されたテキストを表示**
    print("\n🔍 関連するドキュメント（エンベディング検索結果）：")
    for idx, doc in enumerate(relevant_docs):
        print(f"{idx+1}. {doc.page_content}  [Source: {doc.metadata.get('source', '不明')}]")

    # 🔹 **Gemini に質問を送信**
    response = qa_chain.invoke({"question": user_input})

    return response["answer"]

# 8. 実行
print("\n🤖 Gemini の回答:")
print(chat_with_gemini("AIとは何ですか？"))


🤖 Gemini の回答:


C:\Users\yutou\AppData\Local\Temp\ipykernel_24712\417428091.py:50: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = vectorstore.as_retriever().get_relevant_documents(user_input)



🔍 関連するドキュメント（エンベディング検索結果）：
1. AI（人工知能）は、機械が人間のように学習・判断する技術のことです。  [Source: AIの基本]
2. 機械学習は、データからパターンを学び、予測や判断を行う技術です。  [Source: 機械学習]
3. LLM（大規模言語モデル）は、多量のデータを使って訓練された自然言語処理モデルです。  [Source: LLM]
4. GeminiはGoogleが開発した大規模言語モデル（LLM）で、マルチモーダルなタスクに対応可能です。  [Source: Gemini]
AI（人工知能）は、機械が人間のように学習・判断する技術のことです。
